# DynamoDB Store for LangGraph

This notebook demonstrates how to use the DynamoDB store implementation for LangGraph, providing persistent key-value storage with hierarchical namespaces.

## Setup

First, install the required dependencies and configure AWS credentials.

In [ ]:
# Install required packages
# %pip install langgraph-checkpoint-aws boto3

## Basic Usage

In [ ]:
from langgraph_checkpoint_aws import DynamoDBStore

# Create a store instance
store = DynamoDBStore(table_name="my-langgraph-store")

# Setup the table (creates it if it doesn't exist)
store.setup()

## Storing and Retrieving Data

In [ ]:
# Store a document in a hierarchical namespace
store.put(
    ("documents", "user123"),
    "report_1",
    {
        "text": "Machine learning report on customer behavior analysis...",
        "tags": ["ml", "analytics", "report"],
        "author": "data_scientist"
    }
)

# Retrieve the document
item = store.get(("documents", "user123"), "report_1")
print(f"Retrieved item: {item.value}")
print(f"Created at: {item.created_at}")
print(f"Updated at: {item.updated_at}")

## Searching Documents

In [ ]:
# Store multiple documents
store.put(
    ("documents", "user123"),
    "report_2",
    {"text": "Sales report Q1...", "tags": ["sales", "report"], "author": "analyst"}
)

store.put(
    ("documents", "user123"),
    "note_1",
    {"text": "Quick note about meeting...", "tags": ["note"], "author": "data_scientist"}
)

# Search for all documents in the namespace
results = store.search(("documents", "user123"))
print(f"Found {len(results)} documents")
for item in results:
    print(f"  - {item.key}: {item.value['text'][:50]}...")

## Filtering Results

In [ ]:
# Search with filter
results = store.search(
    ("documents", "user123"),
    filter={"author": "data_scientist"}
)

print(f"Found {len(results)} documents by data_scientist")
for item in results:
    print(f"  - {item.key}: {item.value}")

## Using TTL (Time-To-Live)

In [ ]:
from langgraph_checkpoint_aws import DynamoDBStore

# Create a store with TTL configuration
store_with_ttl = DynamoDBStore(
    table_name="my-langgraph-store-ttl",
    ttl={
        "default_ttl": 60,  # 60 minutes default TTL
        "refresh_on_read": True,  # Refresh TTL when items are read
    }
)
store_with_ttl.setup()

# Store a temporary item that will expire after 60 minutes
store_with_ttl.put(
    ("temp", "session_123"),
    "data",
    {"value": "temporary session data"}
)

# Store an item with custom TTL (30 minutes)
store_with_ttl.put(
    ("temp", "session_123"),
    "short_lived",
    {"value": "expires soon"},
    ttl=30
)

print("Items stored with TTL. They will be automatically deleted by DynamoDB after expiration.")

## Listing Namespaces

In [ ]:
# Store items in different namespaces
store.put(("users", "alice"), "prefs", {"theme": "dark"})
store.put(("users", "bob"), "prefs", {"theme": "light"})
store.put(("projects", "ml"), "config", {"model": "gpt-4"})

# List all namespaces
namespaces = store.list_namespaces()
print("All namespaces:")
for ns in namespaces:
    print(f"  - {ns}")

# List namespaces with prefix filter
user_namespaces = store.list_namespaces(prefix=("users",))
print("\nNamespaces starting with 'users':")
for ns in user_namespaces:
    print(f"  - {ns}")

## Deleting Items

In [ ]:
# Delete an item
store.delete(("documents", "user123"), "note_1")

# Verify it's deleted
item = store.get(("documents", "user123"), "note_1")
print(f"Item exists: {item is not None}")  # Should print False

## Using Context Manager

In [ ]:
# Use context manager for automatic cleanup
with DynamoDBStore.from_conn_string("my-langgraph-store") as store:
    store.setup()
    store.put(("test",), "example", {"data": "value"})
    item = store.get(("test",), "example")
    print(f"Retrieved: {item.value}")

## Batch Operations

In [ ]:
from langgraph.store.base import PutOp, GetOp, SearchOp

# Perform multiple operations in a batch
ops = [
    PutOp(("batch",), "item1", {"value": 1}, None, None),
    PutOp(("batch",), "item2", {"value": 2}, None, None),
    PutOp(("batch",), "item3", {"value": 3}, None, None),
]

results = store.batch(ops)
print(f"Batch operation completed: {len(results)} operations")

# Get multiple items
get_ops = [
    GetOp(("batch",), "item1", False),
    GetOp(("batch",), "item2", False),
    GetOp(("batch",), "item3", False),
]

items = store.batch(get_ops)
for item in items:
    if item:
        print(f"  - {item.key}: {item.value}")

## Integration with LangGraph

The DynamoDB store can be used with LangGraph for persistent memory:

In [ ]:
# Example of using DynamoDB store with LangGraph
# (This requires LangGraph to be installed)

# from langgraph.prebuilt import ToolNode
# from langgraph.graph import StateGraph, END
# from langgraph_checkpoint_aws import DynamoDBStore

# store = DynamoDBStore(table_name="langgraph-memory")
# store.setup()

# # Use store for persistent memory across graph executions
# # Store user preferences, conversation history, etc.

print("DynamoDB store can be used for LangGraph persistent memory!")

## Cleanup

In [ ]:
# Note: To delete the DynamoDB table, use AWS Console or boto3 directly
# The store does not provide a method to delete tables to prevent accidental data loss

# import boto3
# dynamodb = boto3.resource('dynamodb')
# table = dynamodb.Table('my-langgraph-store')
# table.delete()

print("Remember to delete DynamoDB tables when no longer needed to avoid charges!")